<a href="https://colab.research.google.com/github/yusuku/DepthEstimationFrom360/blob/main/DepthEstimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os, re
import tensorflow as tf
import numpy as np
import matplotlib.pylab as plt


In [9]:
def vgg16_block(x, num_filters, num_conv_layers):
    for _ in range(num_conv_layers):
        x = tf.keras.layers.Conv2D(num_filters, (3, 3), padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)  # Batch Normalization を追加
        x = tf.keras.layers.ReLU()(x)
    return x

In [10]:
def upsample_block(x, skip_output, num_filters,num_conv_layers):
    x = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding="same")(x)
    x = tf.keras.layers.Concatenate()([x, skip_output])
    x =vgg16_block(x, num_filters, num_conv_layers)
    return x

In [15]:
input_shape = (224, 224, 3)
input_tensor = tf.keras.layers.Input(shape=input_shape)
x = input_tensor

block_outputs = []
#Encord Block0
x = vgg16_block(x, num_filters=64, num_conv_layers=2)
block_outputs.append(x)
#Encord  Block1
x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
x = vgg16_block(x, num_filters=128, num_conv_layers=2)
block_outputs.append(x)
#Encord  Block2
x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
x = vgg16_block(x, num_filters=256, num_conv_layers=3)
block_outputs.append(x)
#Encord  Block3
x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
x = vgg16_block(x, num_filters=512, num_conv_layers=3)
block_outputs.append(x)
#Encord  Block4
x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
x = vgg16_block(x, num_filters=512, num_conv_layers=3)
block_outputs.append(x)

#BottleNeck
x = tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2))(x)
x= tf.keras.layers.Conv2D(1024, (3, 3), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)

#Decord Block4
x=upsample_block(x,block_outputs[4] ,512,3)
#Decord Block3
x=upsample_block(x,block_outputs[3] ,512,3)
#Decord Block2
x=upsample_block(x,block_outputs[2] ,256,3)
#Decord Block1
x=upsample_block(x,block_outputs[1] ,128,2)
#Decord Block0
x=upsample_block(x,block_outputs[0] ,64,2)

depth_output = tf.keras.layers.Conv2D(1, (1, 1), activation="sigmoid", name="depth_map")(x)
uncertainty_output = tf.keras.layers.Conv2D(1, (1, 1), activation="sigmoid", name="uncertainty_map")(x)
normal_output = tf.keras.layers.Conv2D(3, (1, 1), activation="tanh", name="normal_map")(x)




model= tf.keras.Model(inputs=input_tensor, outputs=[depth_output, uncertainty_output, normal_output])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_82 (Conv2D)          (None, 224, 224, 64)         1792      ['input_5[0][0]']             
                                                                                                  
 batch_normalization_95 (Ba  (None, 224, 224, 64)         256       ['conv2d_82[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 re_lu_81 (ReLU)             (None, 224, 224, 64)         0         ['batch_normalization_95

In [18]:
dummy_input = np.random.rand(1, 224, 224, 3)
outputs = model.predict(dummy_input)

for output in outputs:
    print(output.shape)

1/1 [==============================] - 0s 186ms/step
(1, 224, 224, 1)
(1, 224, 224, 1)
(1, 224, 224, 3)
